# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [2]:
# Importando as bibliotecas e pacotes necessários

import pandas as pd
import seaborn as sns
from seaborn import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf


%matplotlib inline

In [3]:
# Carregar os dados
df = pd.read_csv('previsao_de_renda.csv')


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

### 1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.




    



*   Utilizando os recursos do Patsy, coloque as variáveis qualitativas como dummies.
*   Mantenha sempre a categoria mais frequente como casela de referência
*   Avalie os parâmetros e veja se parecem fazer sentido prático.


In [7]:
# Analisando a frequência das categorias em cada variável e escolhendo a categoria mais frequente como referência

for coluna in df.columns[3:]:
    print(coluna, '\n', df[coluna].value_counts(), '\n')


sexo 
 F    10119
M     4881
Name: sexo, dtype: int64 

posse_de_veiculo 
 False    9140
True     5860
Name: posse_de_veiculo, dtype: int64 

posse_de_imovel 
 True     10143
False     4857
Name: posse_de_imovel, dtype: int64 

qtd_filhos 
 0     10376
1      3037
2      1376
3       185
4        17
7         5
14        2
5         2
Name: qtd_filhos, dtype: int64 

tipo_renda 
 Assalariado         7633
Empresário          3508
Pensionista         2582
Servidor público    1268
Bolsista               9
Name: tipo_renda, dtype: int64 

educacao 
 Secundário             8895
Superior completo      5335
Superior incompleto     579
Primário                165
Pós graduação            26
Name: educacao, dtype: int64 

estado_civil 
 Casado      10534
Solteiro     1798
União        1078
Separado      879
Viúvo         711
Name: estado_civil, dtype: int64 

tipo_residencia 
 Casa             13532
Com os pais        675
Governamental      452
Aluguel            194
Estúdio             83
Comu

In [8]:
# Ajustando o modelo de regressão
y, X = patsy.dmatrices(formula_like='''
                                    np.log(renda) ~ sexo
                                                    + posse_de_veiculo
                                                    + posse_de_imovel
                                                    + C(qtd_filhos, Treatment(df['qtd_filhos'].value_counts().idxmax()))
                                                    + C(tipo_renda, Treatment(df['tipo_renda'].value_counts().idxmax()))
                                                    + C(educacao, Treatment(df['educacao'].value_counts().idxmax()))
                                                    + C(estado_civil, Treatment(df['estado_civil'].value_counts().idxmax()))
                                                    + C(tipo_residencia, Treatment(df['tipo_residencia'].value_counts().idxmax()))
                                                    + idade
                                                    + tempo_emprego
                                                    + C(qt_pessoas_residencia, Treatment(df['qt_pessoas_residencia'].value_counts().idxmax()))
                                    ''',
                       data=df)

model = sm.OLS(y, X).fit()

# Imprimindo o resumo do modelo
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.358
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     203.5
Date:                Tue, 03 Oct 2023   Prob (F-statistic):               0.00
Time:                        19:19:14   Log-Likelihood:                -13559.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12392   BIC:                         2.745e+04
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
                                                                                                       coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------


> **ANÁLISE:**

Os parâmetros do modelo de regressão parecem fazer sentido prático em geral, mas há algumas observações importantes a serem feitas:


*   ***Sexo (sexo[T.M])***: O coeficiente positivo sugere que, em média, os homens tendem a ter uma renda maior do que as mulheres, o que pode refletir a disparidade de gênero na renda que é observada na realidade.

*   ***Posse de veículo (posse_de_veiculo[T.True]) e Posse de imóvel (posse_de_imovel[T.True])***: Ambos os coeficientes são positivos, sugerindo que a posse de um veículo ou imóvel está associada a uma maior renda. Isso faz sentido, pois a posse de bens duráveis geralmente requer uma renda maior.

*   ***Quantidade de filhos (qtd_filhos)***: A maioria dos coeficientes para a quantidade de filhos é negativa, sugerindo que ter mais filhos está associado a uma menor renda. Isso pode ser porque ter mais filhos pode levar a maiores despesas e, portanto, menos renda disponível.

*   ***Tipo de renda (tipo_renda)***: Os coeficientes para o tipo de renda variam dependendo do tipo específico de renda. Por exemplo, ser um empresário está associado a uma maior renda, enquanto ser um pensionista está associado a uma menor renda.

*   ***Educação (educacao)***: O coeficiente para o nível de educação primária é positivo, sugerindo que ter pelo menos uma educação primária está associado a uma maior renda. Isso faz sentido, pois um maior nível de educação geralmente leva a melhores oportunidades de emprego e, portanto, a uma maior renda.

### 2. Remova a variável menos significante e analise:
* Observe os indicadores que vimos, e avalie se o modelo melhorou ou
piorou na sua opinião.
* Observe os parâmetros e veja se algum se alterou muito.


In [9]:
# Ajustando o modelo de regressão sem a variável 'tipo_residencia'
model = smf.ols(formula='''
                np.log(renda) ~ sexo
                                + posse_de_veiculo
                                + posse_de_imovel
                                + C(qtd_filhos, Treatment(df['qtd_filhos'].value_counts().idxmax()))
                                + C(tipo_renda, Treatment(df['tipo_renda'].value_counts().idxmax()))
                                + C(educacao, Treatment(df['educacao'].value_counts().idxmax()))
                                + C(estado_civil, Treatment(df['estado_civil'].value_counts().idxmax()))
                                + idade
                                + tempo_emprego
                                + C(qt_pessoas_residencia, Treatment(df['qt_pessoas_residencia'].value_counts().idxmax()))
                ''',
        data=df).fit()

# Imprimindo o resumo do modelo
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.358
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     238.5
Date:                Tue, 03 Oct 2023   Prob (F-statistic):               0.00
Time:                        19:19:21   Log-Likelihood:                -13561.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12397   BIC:                         2.741e+04
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
                                                                                                       coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------



> **ANÁLISE**:

Analisando os resultados da regressão, podemos observar que o R² e o R² ajustado permaneceram os mesmos (0.358 e 0.357, respectivamente) após a remoção da variável `tipo_residencia`. Isso sugere que a remoção dessa variável não teve um impacto significativo na capacidade do modelo de explicar a variabilidade nos dados.

Em relação aos parâmetros do modelo, a maioria deles parece ter permanecido relativamente estável após a remoção da variável `tipo_residencia`.

*Em geral, a remoção da variável `tipo_residencia` não melhorou nem piorou significativamente o modelo.*


### 3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor.



In [15]:
formula = '''
                np.log(renda) ~ sexo
                                + posse_de_veiculo
                                + posse_de_imovel
                                + C(qtd_filhos, Treatment(df['qtd_filhos'].value_counts().idxmax()))
                                + C(tipo_renda, Treatment(df['tipo_renda'].value_counts().idxmax()))
                                + C(educacao, Treatment(df['educacao'].value_counts().idxmax()))
                                + C(estado_civil, Treatment(df['estado_civil'].value_counts().idxmax()))
                                + idade
                                + tempo_emprego
                                + C(qt_pessoas_residencia, Treatment(df['qt_pessoas_residencia'].value_counts().idxmax()))
                '''

# Ajustando o modelo inicial
model = smf.ols(formula=formula, data=df).fit()

In [17]:
# Obtendo os p-values
pvalues = model.pvalues

# Encontrando as variáveis com p-values acima de 0,05
insignificant_vars = pvalues[pvalues > 0.05].index

# Removendo as variáveis insignificantes da fórmula
for var in insignificant_vars:
    formula = formula.replace(' + ' + var, '')

# Ajustando o modelo final
model = smf.ols(formula=formula, data=df).fit()

# Imprimindo o resumo do modelo final
print(model.summary())



                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.358
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     238.5
Date:                Tue, 03 Oct 2023   Prob (F-statistic):               0.00
Time:                        19:27:04   Log-Likelihood:                -13561.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12397   BIC:                         2.741e+04
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
                                                                                                       coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------

> **ANÁLISE**:

Analisando os resultados da regressão, podemos observar que o R² e o R² ajustado permaneceram os mesmos (0.358 e 0.357, respectivamente) após a remoção das variáveis com um valor p maior que 0,05. Isso sugere que a remoção dessas variáveis não teve um impacto significativo na capacidade do modelo de explicar a variabilidade nos dados.

Em relação aos parâmetros do modelo, a maioria deles parece ter permanecido relativamente estável após a remoção das variáveis menos significativas.

*Em geral, a remoção das variáveis menos significativas não melhorou nem piorou significativamente o modelo.*
